In [1]:
%matplotlib notebook

#every line of code in the first 4 boxes is from mirs covid_plot unless stated otherwise 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import gmaps
from ipywidgets.embed import embed_minimal_html
import seaborn as sns
import plotly.express as px
import re
import gmaps.geojson_geometries
from matplotlib.cm import viridis
from matplotlib.cm import Spectral
from matplotlib.colors import to_hex
import country_converter as coco
import os

#this will be will be the stuff ill be using on the animated graphs. im adding them as i see a tutorial
#Not necessarilly deteling them like those on top. In which I can guarantee i am not using half of those.
#I will only cite the ones that ends up working
import matplotlib as cm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas_alive
from celluloid import Camera
from matplotlib.animation import FuncAnimation
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML
from tqdm import tqdm

import bar_chart_race as bcr

In [2]:
covid_stats_df =  pd.read_csv("covid_stats_2020.csv")
covid_stats_df.head()



,standard_names,ISO_2,ISO_3,Population (2020),lat,lng,Confirmed,stringency_index,Infection_rate_f
0,Afghanistan,AF,AFG,39074280,33.939110,67.709953,46274,42.700514,0.11
1,Albania,AL,ALB,2877239,41.153332,20.168331,52004,55.330056,1.81
2,Algeria,DZ,DZA,43984569,28.033886,1.659626,94371,59.981444,0.22
3,Andorra,AD,AND,77287,42.506285,1.521801,7519,40.286864,9.72
4,Angola,AO,AGO,33032075,-11.202692,17.873887,16562,57.541104,0.04


In [3]:
ultimate_no_null_df = pd.read_csv("ultimate_no_null.csv")
ultimate_no_null_df.head(-1)
ultimate_no_null_df['Infection_rate'] = ultimate_no_null_df['Infection_rate'].str.rstrip('%').astype('float')




In [4]:

# ultimate_no_null_df['month'] = ultimate_no_null_df['month'].astype('str')
# modifiednull = ultimate_no_null_df["month"].replace({"1": "1January", "2": "2February",
#                                                     "3": "3March", "4": " 4April",
#                                                     "5": "5May", "6": " 6June",
#                                                     "7": "7July", "8": "8August",
#                                                     "9": "9September", "10": "10October",
#                                                     "11": "11November", "12": "12December"}, inplace=True)


#I wanted to replace the month number to the months, but since the months would become an index, i notice that no matter what
#i did it would alphabeticize the months. Destroying the point of the whole thing. So wwe are force to use months number instead 
#of having their actual names. Uncomment this portion out if you want to see it.

In [5]:
ConfirmedDESC = ultimate_no_null_df.sort_values(by=['Confirmed'], ascending = True).reset_index()
ConfirmedDESC.head(10)

#i will need these to copy and paste the correct file names
#sorting the other data fram. Puprposely not removing the population on this one. Will  do so later if this works
#This should be obvious since this is the beginning. This is not really relevant. Unless its group by countries

,index,standard_names,month,year,Confirmed,stringency_index,lat,lng,Population (2020),Infection_rate,ISO_2,ISO_3
0,0,Afghanistan,1,2020,0,0.000000,33.939110,67.709953,39074280,0.0,AF,AFG
1,1212,Morocco,1,2020,0,0.000000,31.791702,-7.092620,36985624,0.0,MA,MAR
2,595,Estonia,1,2020,0,0.000000,58.595272,25.013607,1326693,0.0,EE,EST
3,587,Eritrea,5,2020,0,93.520000,15.179384,39.782334,3554797,0.0,ER,ERI
4,584,Eritrea,2,2020,0,0.000000,15.179384,39.782334,3554797,0.0,ER,ERI
5,583,Eritrea,1,2020,0,0.000000,15.179384,39.782334,3554797,0.0,ER,ERI
6,1666,Spain,1,2020,0,0.358387,40.463667,-3.749220,46757980,0.0,ES,ESP
7,572,El Salvador,2,2020,0,19.440000,13.794185,-88.896530,6491923,0.0,SV,SLV
8,571,El Salvador,1,2020,0,3.494194,13.794185,-88.896530,6491923,0.0,SV,SLV
9,1679,Sri Lanka,2,2020,0,8.905172,7.873054,80.771797,21428970,0.0,LK,LKA


In [6]:
#Race Bar Chart
Pivot = ultimate_no_null_df.pivot(index='month', columns='standard_names', values='Infection_rate')
Pivot



#its very important to not reset index in this particular chart because if we reset the index month will be treated as a standard name

#I was going to remove the NAN values but the NAN values shows that the countries are not reporting their values for that month.
    #And i was not interested in turn NAN to 0 because that would be giving false data.
    



#In order for the racing bar chart to work, we need first to make turn our df into a wide format.
#So that the month can be the new index, the countries are the new columns, and for this first one will be using the infectionr rate values,
    #The later charts will be using different value, and that will be the only change

#source = https://chrisalbon.com/python/data_wrangling/pandas_long_to_wide/


#changing the data to wide.
# With month as the Index, the countries as the columns, and the Confirmed Cases as the values

standard_names,Afghanistan,Albania,Algeria,Andorra,Angola,Argentina,Australia,Austria,Azerbaijan,Bahrain,...,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
month,,,,,,,,,,,,,,,,,,,,,
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00
3,0.00,0.01,0.00,0.49,0.00,0.00,0.02,0.11,0.00,0.03,...,0.20,0.06,0.01,0.00,0.0,0.00,0.0,0.00,0.00,0.00
4,0.01,0.02,0.01,0.48,0.00,0.01,0.01,0.06,0.01,0.14,...,0.35,0.27,0.02,0.01,0.0,0.00,0.0,0.00,0.00,0.00
5,0.03,0.01,0.01,0.02,0.00,0.03,0.00,0.01,0.04,0.49,...,0.12,0.22,0.01,0.00,0.0,0.00,0.0,0.00,0.01,0.00
6,0.04,0.05,0.01,0.12,0.00,0.11,0.00,0.01,0.12,0.90,...,0.04,0.25,0.00,0.01,0.0,0.02,0.0,0.00,0.00,0.00
7,0.01,0.10,0.04,0.09,0.00,0.28,0.04,0.04,0.14,0.83,...,0.03,0.58,0.01,0.05,0.0,0.04,0.0,0.00,0.02,0.02
8,0.00,0.15,0.03,0.32,0.00,0.50,0.03,0.07,0.04,0.64,...,0.05,0.44,0.01,0.05,0.0,0.10,0.0,0.01,0.03,0.02
9,0.00,0.14,0.02,1.13,0.01,0.74,0.01,0.19,0.04,1.10,...,0.17,0.36,0.01,0.04,0.0,0.10,0.0,0.00,0.01,0.01


In [ ]:
#source = https://www.youtube.com/watch?v=eF9VH4zKoTI
#source2 = https://github.com/dexplo/

race = bcr.bar_chart_race(Pivot, 
                   n_bars = 10,steps_per_period=28,period_length=2000,  
                   title='Top 10 Infection Rate Per Month',     
                   period_summary_func=lambda v, r: {'x': .98, 'y': .2, 's': f'Average Infection Rate: {v.mean():,.0f}', 'ha': 'right', 'size': 11}
                  )

race
#I tried changing the months in to their actual month names instead of their month number, but the because 

#wait about 3 -5 minutes

#I wants smooth animation so I made it to 30fps with each month being on 2 seconds long



C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 128 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 130 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 131 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\bac

C:\Users\danza\.conda\envs\PythonData\lib\site-packages\bar_chart_race\_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "


In [8]:
other = ultimate_no_null_df.pivot(index='month', columns='standard_names', values='stringency_index')




other

standard_names,Afghanistan,Albania,Algeria,Andorra,Angola,Argentina,Australia,Austria,Azerbaijan,Bahrain,...,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
month,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.000000,0.000000,0.000000,3.225484,2.508710,0.000000,0.179355,3.135484,...,2.241613,0.000000,0.000000,3.942258,0.000000,0.000000,2.778387,0.000000,0.000000,1.254194
2,2.010690,3.545172,0.000000,0.000000,4.601379,11.110000,19.440000,2.298621,4.120690,17.528966,...,11.014138,5.368276,0.000000,11.110000,0.000000,0.766207,40.866897,0.000000,5.173448,8.330000
3,40.831935,58.570968,37.486129,21.624839,24.400000,51.702581,38.200323,52.926129,49.762258,49.195484,...,36.633226,45.774516,33.870968,46.657419,16.128387,52.150968,53.492903,18.458710,24.016774,28.284194
4,80.740000,86.665000,86.155333,53.179667,87.529333,98.457000,71.480667,78.642000,89.785333,75.986667,...,79.630000,72.690000,65.985667,91.296000,77.408000,82.410000,88.814000,40.586333,57.686333,87.960000
5,84.260000,87.421613,77.566774,50.896774,78.524839,90.262581,67.352258,60.601935,82.558387,75.000000,...,74.225161,72.690000,61.110000,89.307742,72.762581,84.113871,71.473548,56.957419,49.507742,87.960000
6,80.182667,70.466000,70.554333,42.837000,76.361667,89.136667,55.014333,47.744667,85.743333,75.000000,...,70.866000,70.711333,57.655333,72.066000,66.670000,81.111000,63.331000,58.330000,42.034000,72.715333
7,78.700000,63.856774,72.548387,42.502258,76.450645,91.515484,69.819355,38.682581,95.941290,70.516129,...,65.381290,68.263871,29.392581,69.804194,66.670000,86.740000,58.545806,41.873548,50.930000,76.583548
8,55.408710,58.125484,79.989032,45.370000,78.183548,88.259355,78.213226,35.190000,87.608387,66.754516,...,67.636129,67.130000,29.420968,61.469355,60.934516,85.035806,77.253548,31.480000,50.210000,80.560000
9,23.520000,54.630000,75.930000,48.146667,78.010000,87.960000,74.720000,36.637333,77.192333,61.793000,...,65.569333,64.197667,37.965000,44.383333,22.220000,82.965000,72.178333,29.322667,48.982667,78.457667


In [25]:
race = bcr.bar_chart_race(other, 
                   n_bars = 10,steps_per_period=30,period_length=2000,  
                   title='Top 10 Countries With Highest Controls on Covid', 
                    period_summary_func=lambda v, r: {'x': .98, 'y': .2, 's': f'Average Stringency Index: {v.mean():,.0f}', 'ha': 'right', 'size': 11}
#                    filename='animated\stringan.mp4'
                  )

race

# steps_per_period = 28. I am a video gamer so 60fps is a must for me. But since there



C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 128 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 130 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 131 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\bac

C:\Users\danza\.conda\envs\PythonData\lib\site-packages\bar_chart_race\_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "


In [13]:
cases = ultimate_no_null_df.pivot(index='month', columns='standard_names', values='Confirmed')

cases



standard_names,Afghanistan,Albania,Algeria,Andorra,Angola,Argentina,Australia,Austria,Azerbaijan,Bahrain,...,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
month,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,2.0,8.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,16.0,9.0,0.0,41.0,...,199.0,17.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0
3,174.0,243.0,715.0,376.0,7.0,1054.0,4534.0,10171.0,298.0,526.0,...,132701.0,192152.0,338.0,172.0,0.0,135.0,196.0,0.0,35.0,8.0
4,1952.0,530.0,3290.0,369.0,20.0,3374.0,2207.0,5272.0,1506.0,2473.0,...,236937.0,884047.0,806.0,1867.0,0.0,198.0,58.0,6.0,71.0,32.0
5,13081.0,364.0,5388.0,19.0,59.0,12423.0,436.0,1279.0,3690.0,8358.0,...,78768.0,718241.0,180.0,1584.0,0.0,1177.0,58.0,317.0,951.0,178.0
6,16299.0,1398.0,4513.0,91.0,198.0,47679.0,718.0,1035.0,12030.0,15360.0,...,27677.0,834974.0,113.0,4880.0,0.0,4322.0,27.0,835.0,537.0,413.0
7,5158.0,2741.0,16487.0,70.0,864.0,126772.0,9360.0,3364.0,14354.0,14224.0,...,19577.0,1923898.0,328.0,15506.0,0.0,12742.0,203.0,570.0,4369.0,2578.0
8,1494.0,4237.0,14100.0,251.0,1506.0,226433.0,8539.0,6308.0,4557.0,10990.0,...,33290.0,1463760.0,331.0,17884.0,0.0,28154.0,486.0,2062.0,6134.0,3328.0
9,1109.0,4136.0,7036.0,874.0,2318.0,333266.0,1277.0,17375.0,3794.0,18892.0,...,117763.0,1202896.0,451.0,14824.0,0.0,28394.0,50.0,76.0,2662.0,1341.0


In [28]:
race = bcr.bar_chart_race(cases, 
                   n_bars = 10,steps_per_period=20,period_length=2000,  
                   title='Top 10 COVID COnfirmed Cases Per Month',     
                   period_summary_func=lambda v, r: {'x': .98, 'y': .2, 's': f'Total Cases: {v.sum():,.0f}', 'ha': 'right', 'size': 11}
                  )

race
#There is a summary table that adds up the all the total confirmed cases per month.

C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 128 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 129 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 130 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 131 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\danza\.conda\envs\PythonData\lib\site-packages\matplotlib\bac

C:\Users\danza\.conda\envs\PythonData\lib\site-packages\bar_chart_race\_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "
